In [ ]:
import glob
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
decomp_type = 'PCA' #ICA
n_components = 600

In [ ]:
#Import pre-combined right hemisphere vertex files
vertex_data_right = pd.read_csv("filename.csv", sep=',', index_col=0, header=None)

In [ ]:
#Import pre-combined left hemisphere vertex files
vertex_data_left = pd.read_csv("filename.csv", sep=',', index_col=0, header=None)

In [ ]:
#z-score each feature matrix individually
vertex_data_valid_scaled_right = StandardScaler().fit_transform(vertex_data_right)
vertex_data_valid_scaled_left = StandardScaler().fit_transform(vertex_data_left)
#print(vertex_data_valid_scaled_left.shape)
#print(np.max(vertex_data_valid_scaled))

In [ ]:
#Combine left and right hemispheres
vertex_data_valid_scaled_full = np.concatenate((vertex_data_valid_scaled_left,vertex_data_valid_scaled_right),axis=1)
#print(vertex_data_valid_scaled_full.shape)

In [ ]:
#z-score each subject individually
vertex_data_valid_scaled_full = StandardScaler().fit_transform(np.transpose(vertex_data_valid_scaled_full))
vertex_data_valid_scaled_full = np.transpose(vertex_data_valid_scaled_full)
#print(vertex_data_valid_scaled_full.shape)

In [ ]:
#Setup for ICA or PCA 
if decomp_type == 'PCA':
    comp_model = PCA(n_components=n_components)

if decomp_type == 'ICA':
    comp_model = FastICA(n_components=n_components)
    
all_components = comp_model.fit_transform(vertex_data_valid_scaled_full)#.transpose())

In [ ]:
all_components.shape

In [ ]:
all_weights = comp_model.components_
numweights = all_weights.shape[1]

In [ ]:
#Note: no explained variance for ICA do not run
plt.plot(list(range(0,n_components)),comp_model.explained_variance_ratio_)
plt.xlabel('Component')
plt.ylabel('Explained Variance')
print(comp_model.explained_variance_ratio_[:10])

In [ ]:
#What PC is at the 80% explained variance threshold
n_comps_80p = 0
n_comps_80p_i = 0
while n_comps_80p < .8:
    n_comps_80p = n_comps_80p + comp_model.explained_variance_ratio_[n_comps_80p_i]
    n_comps_80p_i = n_comps_80p_i + 1
print(str(n_comps_80p_i) + " : " + str(n_comps_80p))

In [ ]:
#Export components to single excel file
component_file_location = r'..\filename' + decomp_type +'_'+str(n_components)+'.csv'
np.savetxt(component_file_location, all_components, delimiter=',')

In [ ]:
#Export component weights to individual files
for ix in range(0,all_weights.shape[0]):
    filename_temp_lr = r'..\filename_loading_left_' + + decomp_type + "_" + str(ix+1) + '.txt'
    np.savetxt(filename_temp_lr, all_weights[ix,:numweights//2], delimiter='\n')
    filename_temp_lr = r'..\filename_loading_right_' + str(ix+1) + '.txt'
    np.savetxt(filename_temp_lr, all_weights[ix,numweights//2:], delimiter='\n')